## Settings

In [1]:
import os
import sys
from sys import platform
import pandas as pd
import datetime
BASE_PATH = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(BASE_PATH+'\src' if platform == 'win32' else BASE_PATH+'/src')
from config.global_config import get_global_config
from utils import get_total_iterations

In [2]:
_config = get_global_config()
MODELS = sorted(_config.get('MODELS'))
HP_CONFIG_PATH = os.path.join(BASE_PATH, _config.get('HP_CONFIG_PATH'))
RESULTS_PATH = os.path.join(BASE_PATH, _config.get('RESULT_PATH'))
RESULTS_PATH_SHARED = os.path.join(BASE_PATH, _config.get('RESULT_PATH_SHARED'))
if platform == 'win32':
	HP_CONFIG_PATH = HP_CONFIG_PATH.replace('/', '\\')
	RESULTS_PATH = RESULTS_PATH.replace('/', '\\')
	RESULTS_PATH_SHARED = RESULTS_PATH_SHARED.replace('/', '\\')
EMISSIONS_FILE = 'emissions.csv'
METRICS_FILE = 'metrics.csv'
PARAMS_FILE = 'params.csv'

## Counter #1

Number of runs for each model.

In [3]:
print('\n' + ''.join(['> ' for i in range(33)]))
print(f'\n{"MODEL":<15}{"N_RUN":>10}{"":>10}{"MODEL":<15}{"N_RUN":>10}\n')
print(''.join(['> ' for i in range(33)]))
for n in range(0, len(MODELS), 2):
	n1 = get_total_iterations(HP_CONFIG_PATH + MODELS[n] + ".hyper")
	if n + 1 < len(MODELS):
		n2 = get_total_iterations(HP_CONFIG_PATH + MODELS[n + 1] + ".hyper")
		print(f'{MODELS[n]:<15}{n1:>10}{"":>10}{MODELS[n + 1]:<15}{n2:>10}')
	else:
		print(f'{MODELS[n]:<15}{n1:>10}')


> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

MODEL               N_RUN          MODEL               N_RUN

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
ADMMSLIM            20736          BPR                    27
CDAE                 1296          CFKG                  162
CKE                   243          ConvNCF             20736
DGCF                  243          DMF                    81
DiffRec           2985984          EASE                   54
ENMF                 2592          FISM                   54
GCMC                 1944          ItemKNN               162
KGAT                18225          KGCN                  729
KGIN                 1296          KGNNLS                162
KTUP                23328          LDiffRec        191102976
LINE                  162          LightGCN              216
MCCLK                6480          MKR                  5184
MacridVAE            5184          MultiDAE              162
MultiVAE 

## Counter #2

Number of runs, duration and average duration (on Azure machines) for each model for which the hyperparameter tuning has been executed.

In [4]:
experiments = []
for d in next(os.walk(RESULTS_PATH))[1]:
	if d != 'movielens':
		for m in next(os.walk(RESULTS_PATH + d))[1]:
			experiments.append(os.path.join(d, m))
experiments = sorted(experiments)
results = {
	'emissions': [],
	'duration': []
}

In [5]:
print(''.join(['> ' for i in range(42)]))
print(f'\n{"ID":<5}{"DATASET":<18}{"MODEL":<10}{"RUNs":>8}{"DURATION":>20}{"AVG DURATION":>18}\n')
print(''.join(['> ' for i in range(42)]))
totals = []
for i, v in enumerate(experiments):
	emissions_df = pd.read_csv(os.path.join(RESULTS_PATH, v, EMISSIONS_FILE))
	e = emissions_df[['project_name', 'duration', 'emissions']].groupby(['project_name']).sum()
	if len(e) > 1:
		raise Exception()
	else:
		results['emissions'].append(e.iloc[0]['emissions']*1000)
		results['duration'].append(e.iloc[0]['duration'])
		totals.append(e.iloc[0]['duration'])
		runs = get_total_iterations(HP_CONFIG_PATH + v.split("/")[1] + ".hyper")
		if i > 1 and v.split("/")[0] != experiments[i - 1].split("/")[0]:
			print(f'\n\033[1m{"":<5}{"TOTAL:":<18}{str(datetime.timedelta(seconds=int(sum(totals[:-1])))):>36}\033[0m\n')
			totals = [totals[-1]]
		print(f'{i+1:<5}{v.split("/")[0][:12]:<18}{v.split("/")[1]:<10}{runs:>8}{str(datetime.timedelta(seconds=int(e.iloc[0]["duration"]))):>20}{str(datetime.timedelta(seconds=int(e.iloc[0]["duration"]/runs)))[2:]:>18}')
		if i == len(experiments) - 1:
			print(f'\n\033[1m{"":<5}{"TOTAL:":<18}{str(datetime.timedelta(seconds=int(sum(totals)))):>36}\033[0m\n')
			totals = [totals[-1]]

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

ID   DATASET           MODEL         RUNs            DURATION      AVG DURATION

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
1    amazon_books      BPR             27            19:56:04             44:17
2    amazon_books      CFKG           162     3 days, 8:27:56             29:48
3    amazon_books      CKE            243    3 days, 19:11:17             22:30
4    amazon_books      DMF             81    2 days, 11:52:13             44:20
5    amazon_books      ItemKNN        162            18:54:58             07:00
6    amazon_books      KGNNLS         162     9 days, 1:50:56             20:41
7    amazon_books      LINE           162     4 days, 7:35:54             38:22
8    amazon_books      LightGCN       216   12 days, 18:04:46             25:01
9    amazon_books      MultiDAE       162     5 days, 2:03:14             45:12
10   amazon_books      Spect

## Counter #3

Duration of execution (on MacOs) with default parameters for each model and for each dataset.

In [6]:
experiments = []
for d in next(os.walk(RESULTS_PATH_SHARED))[1]:
	for m in next(os.walk(RESULTS_PATH_SHARED + d))[1]:
		experiments.append(os.path.join(d, m))
experiments = sorted(experiments)
results = {
	'emissions': [],
	'duration': []
}

In [7]:
print(''.join(['> ' for i in range(30)]))
print(f'\n{"ID":<5}{"DATASET":<18}{"MODEL":<12}{"DURATION(h)":>15}\n')
print(''.join(['> ' for i in range(30)]))
for i, v in enumerate(experiments):
	e = pd.read_csv(os.path.join(RESULTS_PATH_SHARED, v, EMISSIONS_FILE))
	results['emissions'].append(e.iloc[0]['emissions']*1000)
	results['duration'].append(e.iloc[0]['duration'])
	print(f'{i+1:<5}{v.split("/")[0][:12]:<18}{v.split("/")[1]:<12}{str(datetime.timedelta(seconds=int(e.iloc[0]["duration"]))):>15}')

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

ID   DATASET           MODEL           DURATION(h)

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
1    amazon_books      BPR                 0:23:36
2    amazon_books      CDAE                0:27:50
3    amazon_books      CFKG                0:08:06
4    amazon_books      CKE                 0:06:27
5    amazon_books      DGCF                7:40:28
6    amazon_books      DMF                 0:40:24
7    amazon_books      DiffRec             1:26:07
8    amazon_books      ENMF                0:26:58
9    amazon_books      FISM                8:40:36
10   amazon_books      GCMC               22:46:54
11   amazon_books      ItemKNN             0:03:03
12   amazon_books      KGCN                0:22:54
13   amazon_books      KGIN                0:07:44
14   amazon_books      KGNNLS              0:22:44
15   amazon_books      KTUP                0:26:26
16   amazon_books      LDiffRec            0:46:42
17   amaz